# Notebook responsável por apresentar os comandos básicos para manipulação de dados no mongo

## Crud com apenas vários documentos:
- Inserir vários documentos **compostos**
- Recuperar vários documentos **compostos**
- Atualizar vários documentos **compostos**
- Excluir vários documentos **compostos**

## Bibliotecas importantes

In [1]:
from pymongo import MongoClient
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker


## Funções uteis

In [2]:
def criar_endereco():
    return {
        'logradouro':fake.street_name(),
        'numero':fake.building_number(),
        'bairro':fake.bairro(),
        'cidade':fake.city(),
        'uf':fake.administrative_unit(),
        'cep':fake.postcode(),
    }

In [3]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(18, 50)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
        'data_nascimento':fake.date()
    }        
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    pessoa['endereco'] = criar_endereco()

    pessoa['telefones'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['telefones'].append(fake.msisdn())

    pessoa['formacoes'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['formacoes'].append(fake.job())
        
        
    return pessoa

## Configurações

In [4]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [5]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_mongo03']

### Exemplo - Criação de vários documentos compostos

In [6]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[{'cpf': '807.542.369-00',
  'data_nascimento': '1983-03-28',
  'email': 'laura.melo@uol.com.br',
  'endereco': {'bairro': 'Serrano',
               'cep': '18245389',
               'cidade': 'da Paz de da Paz',
               'logradouro': 'Chácara de Moura',
               'numero': '1',
               'uf': 'Espírito Santo'},
  'formacoes': ['Gestor ambiental', 'Recreador'],
  'idade': 43,
  'nome': 'Laura Melo',
  'telefones': ['5571948844742', '5531989518820', '5551955298910']},
 {'cpf': '740.125.368-26',
  'data_nascimento': '2018-03-30',
  'email': 'vitor.da cunha@bol.com.br',
  'endereco': {'bairro': 'Santa Lúcia',
               'cep': '51702-390',
               'cidade': 'Freitas',
               'logradouro': 'Praia Thales Fernandes',
               'numero': '615',
               'uf': 'Maranhão'},
  'formacoes': ['Tecnólogo em negócios imobiliários', 'Bartender'],
  'idade': 19,
  'nome': 'Vitor da Cunha',
  'telefones': ['5551996396954']},
 {'cpf': '567.042.138-80',
  '

In [7]:
df = pd.DataFrame.from_records(lista_pessoas)
df

,cpf,nome,idade,data_nascimento,email,endereco,telefones,formacoes
0,807.542.369-00,Laura Melo,43,1983-03-28,laura.melo@uol.com.br,"{'logradouro': 'Chácara de Moura', 'numero': '...","[5571948844742, 5531989518820, 5551955298910]","[Gestor ambiental, Recreador]"
1,740.125.368-26,Vitor da Cunha,19,2018-03-30,vitor.da cunha@bol.com.br,"{'logradouro': 'Praia Thales Fernandes', 'nume...",[5551996396954],"[Tecnólogo em negócios imobiliários, Bartender]"
2,567.042.138-80,Pedro Miguel Carvalho,45,1996-12-22,pedro miguel.carvalho@gmail.com,"{'logradouro': 'Feira Lucca Nogueira', 'numero...","[5571989617889, 5541928387648, 5584917515712]","[Intérprete de Bíblias, Cartógrafo]"
3,694.128.573-73,André Carvalho,33,1998-01-08,andré.carvalho@yahoo.com.br,"{'logradouro': 'Estação de da Luz', 'numero': ...",[5541922886192],"[Continuista, Lancheiro, Cabo]"
4,092.768.543-47,Joana Monteiro,30,2011-04-13,joana.monteiro@gmail.com,"{'logradouro': 'Estação de Moreira', 'numero':...","[5511996405150, 5521902959179]",[Guincheiro]
5,590.746.123-34,Luiz Henrique Rocha,37,2020-03-20,luiz henrique.rocha@uol.com.br,"{'logradouro': 'Recanto de Araújo', 'numero': ...","[5531980340936, 5571920739634]",[Sushiman]
6,217.508.469-85,Natália Peixoto,29,1987-08-09,natália.peixoto@yahoo.com.br,"{'logradouro': 'Loteamento Ana Júlia Duarte', ...","[5521991861139, 5541962001534, 5541947065342]","[Tecnólogo em recursos pesqueiros, Instrumenta..."
7,830.629.714-87,Gustavo Farias,41,2007-09-08,gustavo.farias@yahoo.com.br,"{'logradouro': 'Lagoa Barros', 'numero': '4', ...","[5521924430341, 5541957584099, 5541982813435]",[Designer gráfico]
8,528.903.716-03,Maitê Jesus,34,2001-05-31,maitê.jesus@gmail.com,"{'logradouro': 'Via Azevedo', 'numero': '4', '...","[5581986329539, 5541983093629, 5541968292541]","[Designer de produto ou desenhista industrial,..."
9,321.568.490-05,Sarah Farias,31,1978-03-01,sarah.farias@hotmail.com,"{'logradouro': 'Condomínio de da Luz', 'numero...",[5551956405386],"[Armador, Corretor da bolsa de valores]"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cpf              10 non-null     object
 1   nome             10 non-null     object
 2   idade            10 non-null     int64 
 3   data_nascimento  10 non-null     object
 4   email            10 non-null     object
 5   endereco         10 non-null     object
 6   telefones        10 non-null     object
 7   formacoes        10 non-null     object
dtypes: int64(1), object(7)
memory usage: 768.0+ bytes


### Inserindo os documentos no banco de dados

In [9]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_ids = colecao_pessoa.insert_many(lista_pessoas).inserted_ids
pessoa_ids

[ObjectId('625ff31dbf0e1464748a84bb'),
 ObjectId('625ff31dbf0e1464748a84bc'),
 ObjectId('625ff31dbf0e1464748a84bd'),
 ObjectId('625ff31dbf0e1464748a84be'),
 ObjectId('625ff31dbf0e1464748a84bf'),
 ObjectId('625ff31dbf0e1464748a84c0'),
 ObjectId('625ff31dbf0e1464748a84c1'),
 ObjectId('625ff31dbf0e1464748a84c2'),
 ObjectId('625ff31dbf0e1464748a84c3'),
 ObjectId('625ff31dbf0e1464748a84c4')]

## Recuperando os documentos inseridos

#### Recuperar todos

In [10]:
for p in colecao_pessoa.find({}):
    pp.pprint(p)

{'_id': ObjectId('625ff31dbf0e1464748a84bb'),
 'cpf': '807.542.369-00',
 'data_nascimento': '1983-03-28',
 'email': 'laura.melo@uol.com.br',
 'endereco': {'bairro': 'Serrano',
              'cep': '18245389',
              'cidade': 'da Paz de da Paz',
              'logradouro': 'Chácara de Moura',
              'numero': '1',
              'uf': 'Espírito Santo'},
 'formacoes': ['Gestor ambiental', 'Recreador'],
 'idade': 43,
 'nome': 'Laura Melo',
 'telefones': ['5571948844742', '5531989518820', '5551955298910']}
{'_id': ObjectId('625ff31dbf0e1464748a84bc'),
 'cpf': '740.125.368-26',
 'data_nascimento': '2018-03-30',
 'email': 'vitor.da cunha@bol.com.br',
 'endereco': {'bairro': 'Santa Lúcia',
              'cep': '51702-390',
              'cidade': 'Freitas',
              'logradouro': 'Praia Thales Fernandes',
              'numero': '615',
              'uf': 'Maranhão'},
 'formacoes': ['Tecnólogo em negócios imobiliários', 'Bartender'],
 'idade': 19,
 'nome': 'Vitor da Cunha',

In [11]:
type(colecao_pessoa.find({}))

pymongo.cursor.Cursor

In [12]:
colecao_pessoa.find({}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'techshot_mongo03.pessoas',
  'indexFilterSet': False,
  'parsedQuery': {},
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN', 'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 10,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 10,
  'executionStages': {'stage': 'COLLSCAN',
   'nReturned': 10,
   'executionTimeMillisEstimate': 0,
   'works': 12,
   'advanced': 10,
   'needTime': 1,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 10},
  'allPlansExecution': []},
 'command': {'find': 'pessoas', 'filter': {}, '$db': 'techshot_mongo03'},
 'serverInfo': {'host': '4d7a123a3888',
  'port': 27017,
  'version': '5.0.7',
  'gitVersion': 'b977129dc70eed766cbee7e412d901ee213acb

#### Recuperando por algum outro atributo

In [13]:
df['idade'].mean()

34.2

In [14]:

criterio = {
    'idade': {'$gt':df['idade'].mean()}
}
pp.pprint(criterio)


{'idade': {'$gt': 34.2}}


In [15]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625ff31dbf0e1464748a84bb'),
 'cpf': '807.542.369-00',
 'data_nascimento': '1983-03-28',
 'email': 'laura.melo@uol.com.br',
 'endereco': {'bairro': 'Serrano',
              'cep': '18245389',
              'cidade': 'da Paz de da Paz',
              'logradouro': 'Chácara de Moura',
              'numero': '1',
              'uf': 'Espírito Santo'},
 'formacoes': ['Gestor ambiental', 'Recreador'],
 'idade': 43,
 'nome': 'Laura Melo',
 'telefones': ['5571948844742', '5531989518820', '5551955298910']}


In [16]:
for p in colecao_pessoa.find(criterio):
    pp.pprint(p)

{'_id': ObjectId('625ff31dbf0e1464748a84bb'),
 'cpf': '807.542.369-00',
 'data_nascimento': '1983-03-28',
 'email': 'laura.melo@uol.com.br',
 'endereco': {'bairro': 'Serrano',
              'cep': '18245389',
              'cidade': 'da Paz de da Paz',
              'logradouro': 'Chácara de Moura',
              'numero': '1',
              'uf': 'Espírito Santo'},
 'formacoes': ['Gestor ambiental', 'Recreador'],
 'idade': 43,
 'nome': 'Laura Melo',
 'telefones': ['5571948844742', '5531989518820', '5551955298910']}
{'_id': ObjectId('625ff31dbf0e1464748a84bd'),
 'cpf': '567.042.138-80',
 'data_nascimento': '1996-12-22',
 'email': 'pedro miguel.carvalho@gmail.com',
 'endereco': {'bairro': 'Santa Amelia',
              'cep': '07689825',
              'cidade': 'Ferreira da Praia',
              'logradouro': 'Feira Lucca Nogueira',
              'numero': '53',
              'uf': 'Paraná'},
 'formacoes': ['Intérprete de Bíblias', 'Cartógrafo'],
 'idade': 45,
 'nome': 'Pedro Miguel Carv

#### Recuperando por algum outro atributo

In [17]:
df.loc[5, 'endereco']['cidade']

'Dias'

In [18]:
criterio = {
    'endereco.cidade': {'$eq':df.loc[5, 'endereco']['cidade']}
}
pp.pprint(criterio)


{'endereco.cidade': {'$eq': 'Dias'}}


In [19]:
for p in colecao_pessoa.find(criterio):
    pp.pprint(p)

{'_id': ObjectId('625ff31dbf0e1464748a84c0'),
 'cpf': '590.746.123-34',
 'data_nascimento': '2020-03-20',
 'email': 'luiz henrique.rocha@uol.com.br',
 'endereco': {'bairro': 'São Gabriel',
              'cep': '42353274',
              'cidade': 'Dias',
              'logradouro': 'Recanto de Araújo',
              'numero': '37',
              'uf': 'Maranhão'},
 'formacoes': ['Sushiman'],
 'idade': 37,
 'nome': 'Luiz Henrique Rocha',
 'telefones': ['5531980340936', '5571920739634']}


## Atualizar o documento

In [20]:
df.loc[3, 'cpf']

'694.128.573-73'

In [21]:
criterio = {
    "cpf":df.loc[3, 'cpf']}

novo_valor = {
    "$set": { "endereco.bairro": "Pampulha" }
}
pp.pprint(novo_valor)


{'$set': {'endereco.bairro': 'Pampulha'}}


In [22]:
r = colecao_pessoa.update_many(criterio, novo_valor)
r

In [23]:
# ref.: https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html
r.matched_count

1

In [24]:
r.modified_count

1

In [25]:
r.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [26]:
r.upserted_id

In [27]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625ff31dbf0e1464748a84be'),
 'cpf': '694.128.573-73',
 'data_nascimento': '1998-01-08',
 'email': 'andré.carvalho@yahoo.com.br',
 'endereco': {'bairro': 'Pampulha',
              'cep': '26703-049',
              'cidade': 'Lima do Norte',
              'logradouro': 'Estação de da Luz',
              'numero': '29',
              'uf': 'Mato Grosso'},
 'formacoes': ['Continuista', 'Lancheiro', 'Cabo'],
 'idade': 33,
 'nome': 'André Carvalho',
 'telefones': ['5541922886192']}


## Remover todos os documentos

In [28]:
criterio = {}
pp.pprint(criterio)

{}


In [29]:
r = colecao_pessoa.delete_many(criterio)

In [30]:
r.raw_result

{'n': 10, 'ok': 1.0}

In [31]:
p = colecao_pessoa.find_one({})
pp.pprint(p)

None
